In [1]:
!nvidia-smi


'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [5]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: accelerate 1.7.0
Uninstalling accelerate-1.7.0:
  Successfully uninstalled accelerate-1.7.0
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached accelerate-1.7.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached accelerate-1.7.0-py3-none-any.whl (362 kB)

   ---------------------------------------- 0/2 [accelerate]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transformers]
   -------------------- ------------------- 1/2 [transform

In [6]:
!pip install evaluate


In [7]:
!pip install torch --upgrade

In [9]:
!pip install torch==2.1.2 torchvision torchaudio


In [10]:
!pip install torch==2.1.2 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu



Looking in indexes: https://download.pytorch.org/whl/cpu


In [11]:
import torch
print(torch.__version__)


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.6 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\GURUDAS N VIJAYAN\anaconda3\envs\HFaceTextS\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\GURUDAS N VIJAYAN\anaconda3\envs\HFaceTextS\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\GURUDAS N VIJAYAN\anaconda3\envs\HFaceTextS\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\GURUDAS N VIJAYAN\anaconda3\envs\HFaceTextS\lib\site-packages\traitlets\con

2.1.2+cpu


In [13]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
from evaluate import load as load_metric

import pandas as pd


from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\GURUDAS N
[nltk_data]     VIJAYAN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model_ckpt = 'google/pegasus-cnn_dailymail'

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Error while downloading from https://cdn-lfs.hf.co/google/pegasus-cnn_dailymail/89463bc493ce252a2919b01ad3d00bae933f079d0cf279cf96c33312dc63a249?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1747563611&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0NzU2MzYxMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9nb29nbGUvcGVnYXN1cy1jbm5fZGFpbHltYWlsLzg5NDYzYmM0OTNjZTI1MmEyOTE5YjAxYWQzZDAwYmFlOTMzZjA3OWQwY2YyNzljZjk2YzMzMzEyZGM2M2EyNDk%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=Om9USgQgXOs7R%7EW5Kw19ngSdeCRG3kWV3WNNO6RHTJp%7EN921cA9HGT4GjwH4azbUUoPYZfrtML0Vgf8vVzes8Vl31FfxFUaDwoIMru4it234datzCyHf958Xx6rcDPxBfvlLRVJEF1QetlfneOYuZCc2J9qaWqM%7EkFKLB-lL%7EZo2DVVK77D27MzG7xaWX7F7%7EeYTJ9oOObAxI61NOCn7AQY%7EUs%7EiGHu52rnAptlibXYXHwJhtoAVxQvujq8VrfjD%7EU7TDaV2K2kdkDl7ZwBMDJoVO7p1MGkWAcx4NvG4l8WWpzsTsSHSTE5CitQ%7EFxNIpOxdIAAuNUFwJzZpd4uv4Q__&Key-Pair-Id=K3RP

In [19]:
device

'cpu'

In [20]:
# download and unzip data

!wget https://github.com/dystinktbeatz/Text-summarizer-project/raw/refs/heads/main/samsum_dataset_G.zip
!unzip summarizer-data.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.
'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import urllib.request
import zipfile
import os

# Download the file
url = "https://github.com/dystinktbeatz/Text-summarizer-project/raw/refs/heads/main/samsum_dataset_G.zip"
local_zip_path = "samsum_dataset_G.zip"

urllib.request.urlretrieve(url, local_zip_path)
print("✅ Downloaded successfully!")

# Extract the zip file
with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
    zip_ref.extractall("extracted_data")  # choose your target folder

print("✅ Unzipped successfully!")


✅ Downloaded successfully!
✅ Unzipped successfully!


In [22]:

print(os.listdir())


['01_data_ingestion.ipynb', '02_data_validation.ipynb', 'extracted_data', 'samsum_dataset_G.zip', 'Text_Summarization.ipynb', 'trials.ipynb']


In [23]:
import os
print(os.listdir("extracted_data"))


['samsum-test.csv', 'samsum-train.csv', 'samsum-validation.csv', 'samsum_dataset']


In [24]:


path = "extracted_data/samsum_dataset"
print("Is it a file?", os.path.isfile(path))
print("Is it a directory?", os.path.isdir(path))
print("Contents (if directory):", os.listdir(path) if os.path.isdir(path) else "N/A")


Is it a file? False
Is it a directory? True
Contents (if directory): ['dataset_dict.json', 'test', 'train', 'validation']


In [25]:
from datasets import load_from_disk
dataset_samsum = load_from_disk("extracted_data/samsum_dataset")
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [26]:
split_lengths = [len(dataset_samsum[split])for split in dataset_samsum]

print(f'Split lengths: {split_lengths}')
print(f'Features: {dataset_samsum["train"].column_names}')
print("\nDialogue:")

print(dataset_samsum['test'][1]['dialogue'])

print("\nSummary:")
print(dataset_samsum['test'][1]['summary'])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [27]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

In [28]:
dataset_samsun_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\GURUDAS N VIJAYAN\anaconda3\envs\HFaceTextS\lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 8138.37 examples/s]


In [29]:
#  Training

from transformers import DataCollatorForSeq2Seq

seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [32]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    #evaluation_strategy='epoch',
    #logging_delay=500,
    save_steps=1e6,
    gradient_accumulation_steps=16,
    #no_deprecation_warning=True,
    report_to="none"
    #save_total_limit=1
)

In [33]:
# Define a Trainer instance
trainer = Trainer(
    model=model_pegasus,
    args=trainer_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_samsun_pt['test'], #!!! always use 'train' here. test is used because the size is small here
    eval_dataset=dataset_samsun_pt['validation']


)


C:\Users\GURUDAS N VIJAYAN\AppData\Local\Temp\ipykernel_25440\1708947673.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [34]:
trainer.train()

Step,Training Loss
10,3.110100
20,3.046600
30,3.161200
40,2.989800
50,2.852400


c:\Users\GURUDAS N VIJAYAN\anaconda3\envs\HFaceTextS\lib\site-packages\transformers\modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=51, training_loss=3.0227330572464886, metrics={'train_runtime': 2114.0149, 'train_samples_per_second': 0.387, 'train_steps_per_second': 0.024, 'total_flos': 313450454089728.0, 'train_loss': 3.0227330572464886, 'epoch': 0.9963369963369964})

In [35]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]

def calculate_metric_on_test_ds(dataset, metric, model, tokenizer,
                               batch_size=16, device=device,
                               column_text="article",
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):

        inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                        padding="max_length", return_tensors="pt")

        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                                   attention_mask=inputs["attention_mask"].to(device),
                                          length_penalty=0.8, num_beams=8, max_length=128)
        '''parameter for length penalty ensures that the model does not generate sequences that are too long.'''

        # Finally, we decode the generated texts,
        # replace the  token, and add the decoded texts with the reference to the metric.

        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                clean_up_tokenization_spaces=True)
               for s in summaries]

        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]


        metric.add_batch(predictions=decoded_summaries, references=target_batch)

        #Finally compute and return the ROUGE scores
        score = metric.compute()
    return score

In [36]:
rouge_names = ["rouge1","rouge2","rougeL","rougeeLsum"]
rouge_metric  = load_metric('rouge')

In [39]:
print(score.keys())



dict_keys(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])


In [ ]:
rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']



In [40]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size=2,
    column_text='dialogue',
    column_summary='summary'
)

rouge_names = ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=['pegasus'])


100%|██████████| 5/5 [03:16<00:00, 39.37s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.036334,0.0,0.036334,0.036334


In [38]:
score = calculate_metric_on_test_ds(
    dataset_samsum['test'][0:10],
    rouge_metric,
    trainer.model,
    tokenizer,
    batch_size=2,
    column_text='dialogue',
    column_summary='summary'
)

rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

pd.DataFrame(rouge_dict, index=[f'pegasus'])


100%|██████████| 5/5 [02:57<00:00, 35.46s/it]


KeyError: 'rougeeLsum'

In [46]:
# Save model

model_pegasus.save_pretrained("pegasus-samsum-model")

In [47]:
# Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer\\tokenizer_config.json',
 'tokenizer\\special_tokens_map.json',
 'tokenizer\\spiece.model',
 'tokenizer\\added_tokens.json',
 'tokenizer\\tokenizer.json')

In [52]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tokenizer")


In [53]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}

sample_text = dataset_samsum["test"][0]["dialogue"]

reference = dataset_samsum["test"][0]["summary"]

pipe = pipeline("summarization", model="pegasus-samsum-model",tokenizer=tokenizer)

print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Device set to use cpu
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
